# Transfer_learning 전이학습
: 기존 학습되어있는 모델을 변경

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Load model

In [3]:
model = models.resnet50(pretrained=True)

In [4]:
# Freeze model param
for param in model.parameters():
    param.requires_grad = False


In [5]:
NUM_CLASSES = 10
model.fc = nn.Linear(in_features=model.fc.in_features,
                     out_features=NUM_CLASSES)


## Load data

In [6]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.3,0.3,0.3))
])

In [7]:
train_datasets = torchvision.datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
test_datasets = torchvision.datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transform)


train_loader = DataLoader(train_datasets, batch_size=32, shuffle=True)
test_loader = DataLoader(test_datasets, batch_size=32, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10\cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


## Set model, loss optim

In [8]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training

In [9]:
NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    train_loss = 0.0
    model.train()
    for images, labels in train_loader:
        # to device
        images = images.to(device)
        labels = labels.to(device)

        #optim zero
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()*images.size(0)

    train_loss /=len(train_loader.dataset)
    print(f"Epoch: [{epoch+1}/{NUM_EPOCHS}], Loss: {train_loss}")

Epoch: [1/10], Loss: 0.762169259929657


KeyboardInterrupt: 

## Evaluation

In [ ]:
model.eval()
correct = 0.0
total = 0.0

with torch.no_grad():
    for images, labels in train_loader:
        # to deivce
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predeicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predeicted == labels).sum().item()

accuracy = 100* correct /total
print(f"Test Accuracy: {accuracy:.2f}%")